In [2]:
# First, install and import all required libraries
import chromadb
from sentence_transformers import SentenceTransformer
import pandas as pd

print("✅ Libraries imported successfully!")

# Initialize ChromaDB client and collection
def initialize_chromadb():
    """Initialize ChromaDB client and get the collection"""
    try:
        chroma_client = chromadb.PersistentClient(path="./chroma_db")
        collection = chroma_client.get_collection("video_embeddings")
        print("✅ ChromaDB initialized successfully!")
        print(f"📊 Collection has {collection.count()} videos")
        return collection
    except Exception as e:
        print(f"❌ Error initializing ChromaDB: {e}")
        print("\n🔧 Please make sure:")
        print("   1. You have run the previous cells that added data to ChromaDB")
        print("   2. The './chroma_db' folder exists")
        print("   3. The collection 'video_embeddings' exists")
        return None

# Initialize the collection
collection = initialize_chromadb()

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded successfully!")

✅ Libraries imported successfully!
✅ ChromaDB initialized successfully!
📊 Collection has 614 videos
✅ Embedding model loaded successfully!


In [3]:
def get_top_5_relevant_videos(user_query):
    """
    Fetch top 5 most relevant videos based on user query
    """
    if collection is None:
        print("❌ ChromaDB not initialized. Please run the previous cell first.")
        return []
    
    print(f"🎯 Processing query: '{user_query}'")
    print("=" * 60)
    
    # Step 1: Convert query to embeddings
    print("🔮 Converting query to embeddings...")
    query_embedding = model.encode([user_query]).tolist()[0]
    print(f"✅ Query embedding generated (dimensions: {len(query_embedding)})")
    
    # Step 2: Perform similarity search on vector database
    print("🔍 Searching for similar videos...")
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5,
        include=['metadatas', 'documents', 'distances']
    )
    
    # Step 3: Process and display results
    print(f"\n🎉 Found {len(results['metadatas'][0])} relevant videos:")
    print("=" * 60)
    
    top_videos = []
    
    for i, (metadata, distance) in enumerate(zip(
        results['metadatas'][0],
        results['distances'][0]
    )):
        # Calculate similarity score (1 - distance) since ChromaDB returns distances
        similarity_score = 1 - distance
        
        # Get video details
        video_id = metadata.get('video_id', 'N/A')
        title = metadata.get('title', 'N/A')
        channel_name = metadata.get('channel_title', 'N/A')
        
        # Store result
        video_info = {
            'rank': i + 1,
            'video_id': video_id,
            'title': title,
            'channel_name': channel_name,
            'similarity_score': round(similarity_score, 4)
        }
        top_videos.append(video_info)
        
        # Display result
        print(f"\n🏆 Rank {i+1}:")
        print(f"   📹 Video ID: {video_id}")
        print(f"   🎬 Title: {title}")
        print(f"   📺 Channel: {channel_name}")
        print(f"   ⭐ Similarity Score: {similarity_score:.4f}")
        print("-" * 50)
    
    return top_videos

In [4]:
# Test the function
def test_query_system():
    """
    Test the query system with example queries
    """
    if collection is None:
        print("❌ ChromaDB not initialized. Please run the first cell.")
        return
    
    print("🚀 Testing Query System")
    print("=" * 60)
    
    test_queries = [
        "ancient mysteries and secrets",
        "treasure hunt shipwreck",
        "scientific discoveries",
        "mountain exploration"
    ]
    
    for query in test_queries:
        results = get_top_5_relevant_videos(query)
        print("\n" + "=" * 80 + "\n")

# Run test
test_query_system()

🚀 Testing Query System
🎯 Processing query: 'ancient mysteries and secrets'
🔮 Converting query to embeddings...
✅ Query embedding generated (dimensions: 384)
🔍 Searching for similar videos...

🎉 Found 5 relevant videos:

🏆 Rank 1:
   📹 Video ID: DvxiRgk2RZI
   🎬 Title: Cleopatra’s Tomb Found? Egypt’s Greatest Discovery in 2,000 Years
   📺 Channel: BRIGHT SIDE
   ⭐ Similarity Score: 0.0155
--------------------------------------------------

🏆 Rank 2:
   📹 Video ID: BGjRfCHkoVY
   🎬 Title: Weirdest Things People Actually Found in the Desert
   📺 Channel: BRIGHT SIDE
   ⭐ Similarity Score: -0.0876
--------------------------------------------------

🏆 Rank 3:
   📹 Video ID: M0dANf5hmC8
   🎬 Title: Shocking Discovery in Spain: Impossible Engineering from 6,000 Years Ago
   📺 Channel: BRIGHT SIDE
   ⭐ Similarity Score: -0.2079
--------------------------------------------------

🏆 Rank 4:
   📹 Video ID: wOnm_5Qwlb0
   🎬 Title: This Pyramid Was Locked for 4,000 Years - Until Now
   📺 Channel: B